In [1]:
#!pip install langchain==0.2.14
#!pip install langchain_community==0.2.14
#!pip install pydantic==1.10.8
#!pip install gradio==4.44.1
!pip install -U openai langchain langchain-openai gradio ipywidgets

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from ipywidgets import FileUpload
from IPython.display import display

uploader = FileUpload(accept='.pdf', multiple=True)
display(uploader)

FileUpload(value=(), accept='.pdf', description='Upload', multiple=True)

In [3]:
print(uploader.value[0])

{'name': '소나기.pdf', 'type': 'application/pdf', 'size': 123544, 'content': <memory at 0x7ff612f36140>, 'last_modified': datetime.datetime(2025, 1, 19, 23, 21, 7, 65000, tzinfo=datetime.timezone.utc)}


In [4]:
bytes_data = uploader.value[0].content.tobytes()

In [5]:
print(type(bytes_data))

<class 'bytes'>


In [6]:
import os

def save_uploaded_pdf(uploader):
    uploaded_file = list(uploader.value)[0]
    file_name = uploaded_file['name']
    save_path = os.path.join(os.getcwd(), file_name)
    with open(save_path, 'wb') as f:
        f.write(uploaded_file['content'])
    
    print(f'Uploaded and saved: {file_name}')
    return file_name

file_name = save_uploaded_pdf(uploader)

Uploaded and saved: 소나기.pdf


In [7]:
!pip install pypdf

Defaulting to user installation because normal site-packages is not writeable


In [8]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(file_name)
pages = loader.load_and_split()

In [9]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader(file_name)
documents = loader.load()

from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
len(docs)

7

In [10]:
docs[:3]

[Document(metadata={'source': '소나기.pdf', 'page': 0}, page_content='- 1 -\n소나기황순원소년은 개울가에서 소녀를 보자 곧 윤 초시네 증손녀(曾孫女)딸이라는 걸 알 수 있었다. 소녀는 개울에다 손을 잠그고 물장난을 하고 있는 것이다. 서울서는 이런 개울물을 보지 못하기나 한 듯이.벌써 며칠째 소녀는, 학교에서 돌아오는 길에 물장난이었다. 그런데, 어제까지 개울 기슭에서 하더니, 오늘은 징검다리 한가운데 앉아서 하고 있다.소년은 개울둑에 앉아 버렸다. 소녀가 비키기를 기다리자는 것이다.요행 지나가는 사람이 있어, 소녀가 길을 비켜 주었다.다음 날은 좀 늦게 개울가로 나왔다.이 날은 소녀가 징검다리 한가운데 앉아 세수를 하고 있었다. 분홍 스웨터 소매를 걷어올린 목덜미가 마냥 희었다.한참 세수를 하고 나더니, 이번에는 물 속을 빤히 들여다 본다. 얼굴이라도 비추어 보는 것이리라. 갑자기 물을 움켜 낸다. 고기 새끼라도 지나가는 듯.소녀는 소년이 개울둑에 앉아 있는 걸 아는지 모르는지 그냥 날쌔게 물만 움켜 낸다. 그러나, 번번이 허탕이다. 그대로 재미있는 양, 자꾸 물만 움킨다. 어제처럼 개울을 건너는 사람이 있어야 길을 비킬 모양이다.그러다가 소녀가 물 속에서 무엇을 하나 집어 낸다. 하얀 조약돌이었다. 그리고는 벌떡 일어나 팔짝팔짝 징검다리를 뛰어 건너간다.다 건너가더니만 홱 이리로 돌아서며,“이 바보.”조약돌이 날아왔다.소년은 저도 모르게 벌떡 일어섰다.단발 머리를 나풀거리며 소녀가 막 달린다. 갈밭 사잇길로 들어섰다. 뒤에는 청량한 가을 햇살 아래 빛나는 갈꽃뿐.이제 저쯤 갈밭머리로 소녀가 나타나리라. 꽤 오랜 시간이 지났다고 생각됐다. 그런데도 소녀는 나타나지 않는다. 발돋움을 했다. 그러고도 상당한 시간이 지났다고 생각됐다.저 쪽 갈밭머리에 갈꽃이 한 옴큼 움직였다. 소녀가 갈꽃을 안고 있었다. 그리고, 이제는 천천한 걸음이었다. 유난히 맑은 가을 햇살이 소녀의 갈꽃머리에서 반짝거렸다. 소녀 아닌 갈

In [11]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.9)

In [12]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
#embeddings = HuggingFaceEmbeddings()

from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import FAISS

index = VectorstoreIndexCreator(
    vectorstore_cls=FAISS,
    embedding=embeddings,
    ).from_loaders([loader])

# 파일로 저장
index.vectorstore.save_local("shower")

/usr/lib/python3.10/importlib/__init__.py:126: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  return _bootstrap._gcd_import(name[level:], package, level)


In [13]:
index.query("소년은 어디에서 처음 소녀를 만났나?", llm=chat, verbose=True)



> Entering new RetrievalQA chain...

> Finished chain.


'소년은 개울가에서 처음 소녀를 만났습니다.'

In [14]:
index.query("소년은 소녀에게 무엇을 주려고 했나?", llm=chat, verbose=True)



> Entering new RetrievalQA chain...

> Finished chain.


'소년은 소녀에게 덕쇠 할아버지네 호두를 맛보여주고 싶어했습니다.'

In [15]:
index.query("소녀는 소년에게 무엇을 주었나?", llm=chat, verbose=True)



> Entering new RetrievalQA chain...

> Finished chain.


'소녀는 소년에게 하얀 조약돌을 주었습니다.'

In [16]:
index.query("소녀와 윤초시는 어떤 관계인가?", llm=chat, verbose=True)



> Entering new RetrievalQA chain...

> Finished chain.


'소녀는 윤 초시의 증손녀(曾孫女)입니다.'